In [1]:
import cv2
import numpy as np
import os
import mediapipe as mp # MediaPipeのコアモジュール
import json

# 設定
DATA_PATH = os.path.join(os.getcwd(), 'MP_Data_JSON')
#actions = np.array(['heel_hook', 'deadpoint', 'dyno','cross_move'])
actions =np.array(['deadpoint','dyno','cross_move'])
no_videos = 30 #90    
sequence_length = 180

# --- ★変更点: MediaPipe Poseモデルの準備 ---
mp_pose = mp.solutions.pose # HolisticからPoseに変更
mp_drawing = mp.solutions.drawing_utils
# mp_drawing_styles = mp.solutions.drawing_styles # 必要であればこちらも利用

# 取得したい姿勢ランドマークのインデックスを指定
#DESIRED_POSE_LANDMARKS = [11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32] # これはユーザー指定のものを維持
#DESIRED_POSE_LANDMARKS = [11,12,13,14,15,16,23,24,25,26,27,28] #Openposeと同じランドマーク(顔以外)
DESIRED_POSE_LANDMARKS = [11,12,13,14,15,16,23,24,25,26,27,28,29,30,31,32] #Openposeと同じ。つまさき・かかとも含む
#DESIRED_POSE_LANDMARKS = [11,12,13,14,15,16] #Openposeと同じランドマーク(両腕)
#DESIRED_POSE_LANDMARKS = [23,24,25,26,27,28] #Openposeと同じランドマーク(両足)
#DESIRED_POSE_LANDMARKS = [12,14,16] #Openposeと同じランドマーク(右腕)
#DESIRED_POSE_LANDMARKS = [11,13,15] #Openposeと同じランドマーク(左腕)
#DESIRED_POSE_LANDMARKS = [24,26,28] #Openposeと同じランドマーク(右足)
#DESIRED_POSE_LANDMARKS = [23,25,27] #Openposeと同じランドマーク(左足)

def multiple_detection(image, model): # 関数名はそのまま使えます
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image) # ここで渡されるmodelがPoseモデルのインスタンスになる
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

def draw_styled_landmarks(image, results):
    image_height, image_width, _ = image.shape

    # 姿勢のキーポイントの描画
    if results.pose_landmarks:
        # 点の描画
        for idx in DESIRED_POSE_LANDMARKS:
            if idx < len(results.pose_landmarks.landmark):
                landmark = results.pose_landmarks.landmark[idx]
                if landmark.visibility < 0.1: # visibilityチェックも有効
                
                     continue
                cx = int(landmark.x * image_width)
                cy = int(landmark.y * image_height)
                cv2.circle(image, (cx, cy), 3, (0, 0, 255), cv2.FILLED)

        # 線の描画 (POSE_CONNECTIONS を元に、指定ランドマーク間のみ)
        # --- ★変更点: mp_holistic.POSE_CONNECTIONS から mp_pose.POSE_CONNECTIONS へ ---
        if mp_pose.POSE_CONNECTIONS:
            for connection in mp_pose.POSE_CONNECTIONS:
                start_idx = connection[0]
                end_idx = connection[1]

                if start_idx in DESIRED_POSE_LANDMARKS and end_idx in DESIRED_POSE_LANDMARKS:
                    if start_idx < len(results.pose_landmarks.landmark) and \
                       end_idx < len(results.pose_landmarks.landmark):

                        start_landmark = results.pose_landmarks.landmark[start_idx]
                        end_landmark = results.pose_landmarks.landmark[end_idx]

                        if start_landmark.visibility < 0.1 or end_landmark.visibility < 0.1:
                             continue

                        start_point = (int(start_landmark.x * image_width), int(start_landmark.y * image_height))
                        end_point = (int(end_landmark.x * image_width), int(end_landmark.y * image_height))
                        cv2.line(image, start_point, end_point, (51, 255, 51), 2)

    # Poseモデルでは顔や手のランドマークは基本的には出力されないので、
    # results.face_landmarks などにアクセスする部分は不要になります。

def extract_keypoints(results):
    keypoints_data = {}

    if results.pose_landmarks:
        pose_landmarks_data = []
        for i, res in enumerate(results.pose_landmarks.landmark):
            if i in DESIRED_POSE_LANDMARKS:
                pose_landmarks_data.append({
                    "id": i,
                    "x": res.x,
                    "y": res.y,
                    "z": res.z,
                    "visibility": res.visibility
                })
        keypoints_data["pose"] = pose_landmarks_data
    else:
        keypoints_data["pose"] = []


    return keypoints_data

# データの収集
cv2.namedWindow('OpenCV Feed', cv2.WINDOW_NORMAL)
cv2.resizeWindow('OpenCV Feed', (1280, 720))

# --- ★変更点: HolisticからPoseモデルのインスタンス作成へ ---
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
with mp_pose.Pose(min_detection_confidence=0.1, min_tracking_confidence=0.1, model_complexity=2) as pose_model: # model_complexity は 0, 1, 2 で調整可能
    for action in actions:
        action_path = os.path.join(DATA_PATH, action)
        if not os.path.exists(action_path):
            os.makedirs(action_path)

        for video_num in range(1, no_videos + 1):
            video_file_path = os.path.join(r"C:\Users\admin\Downloads\my video testdata\data1",action + str(video_num) + '.mp4')
            #video_file_path = os.path.join(r"C:\Users\admin\Downloads\youtubeテストデータ", action + str(video_num) + '.mp4')
            #video_file_path = os.path.join(r'C:\Users\admin\Downloads\senteivideo(1)', action + str(video_num) + '.mp4')
            #video_file_path = os.path.join(r"C:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\senteivideo", action + str(video_num) + '.mp4')
            cap = cv2.VideoCapture(video_file_path)

            video_data_save_path = os.path.join(action_path, str(video_num))
            if not os.path.exists(video_data_save_path):
                os.makedirs(video_data_save_path)

            print(f'Processing video: {video_file_path}')
            frames_collected_for_current_video = 0

            for frame_num in range(sequence_length):
                ret, frame = cap.read()
                if not ret:
                    print(f"Warning: Video {video_file_path} ended before collecting {sequence_length} frames. Collected {frame_num} frames.")
                    break

                # --- ★変更点: holistic を pose_model に変更 ---
                image, results = multiple_detection(frame, pose_model)

                draw_styled_landmarks(image, results)

                cv2.putText(image, f'Collecting frames for {action} - Video {video_num} - Frame {frame_num + 1}/{sequence_length}',
                            (15, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2, cv2.LINE_AA)
                cv2.imshow('OpenCV Feed', image)

                keypoints = extract_keypoints(results)
                json_path = os.path.join(video_data_save_path, f'{frame_num}.json')
                with open(json_path, 'w') as f:
                    json.dump(keypoints, f, indent=4)

                frames_collected_for_current_video +=1

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
            
            cap.release()
            if frames_collected_for_current_video < sequence_length:
                print(f"Warning: For video {video_file_path}, only {frames_collected_for_current_video}/{sequence_length} frames were saved to {video_data_save_path}.")

            if cv2.waitKey(10) & 0xFF == ord('q'): # ループを抜けるための処理
                break
        if cv2.waitKey(10) & 0xFF == ord('q'): # ループを抜けるための処理
            break

cv2.destroyAllWindows()

Processing video: C:\Users\admin\Downloads\my video testdata\data1\deadpoint1.mp4
Processing video: C:\Users\admin\Downloads\my video testdata\data1\deadpoint2.mp4
Processing video: C:\Users\admin\Downloads\my video testdata\data1\deadpoint3.mp4
Processing video: C:\Users\admin\Downloads\my video testdata\data1\deadpoint4.mp4
Processing video: C:\Users\admin\Downloads\my video testdata\data1\deadpoint5.mp4
Processing video: C:\Users\admin\Downloads\my video testdata\data1\deadpoint6.mp4
Processing video: C:\Users\admin\Downloads\my video testdata\data1\deadpoint7.mp4
Processing video: C:\Users\admin\Downloads\my video testdata\data1\deadpoint8.mp4
Processing video: C:\Users\admin\Downloads\my video testdata\data1\deadpoint9.mp4
Processing video: C:\Users\admin\Downloads\my video testdata\data1\deadpoint10.mp4
Processing video: C:\Users\admin\Downloads\my video testdata\data1\deadpoint11.mp4
Processing video: C:\Users\admin\Downloads\my video testdata\data1\deadpoint12.mp4
Processing vi